##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Onceden islenmis metin ile metin siniflandirilmasi: Film yorumlari

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


Bu kitapcikta film yorumlarini metinlerin icerigini kullanarak *olumlu* ve *olumsuz* olarak siniflandiracagiz. Bu onemli ve yogun olarak kullanilan *ikili*- ya da iki-yonlu-siniflandirma makine ogrenmesi sorununun bir ornegidir. 

[Internet Movie Database](https://www.imdb.com/) sitesinden alinan 50,000 film yorumu metnini barindiran [IMDB veri setini](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) kullanacagiz. Bunlarin 25,000 adetini egitim icin kalan 25,000 adeti ise test icin ayiralim. Test ve egitim setleri *dengelidir*, yani esit sayida pozitif ve negatif yorum bulundururlar.

Bu kitapcik yuksek-seviye API [tf.keras](https://www.tensorflow.org/guide/keras) kullanarak TensorFlow icinde modeli olusturur ve egitir. `tf.keras` kullanan daha ileri seviyede metin siniflandirmasi rehberi icin [buraya bakiniz](https://developers.google.com/machine-learning/guides/text-classification/).

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version sadece Colab icinde bulunur.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## IMDB veri setini indirelim

IMDB veri seti TensorFlow paketi ile beraber gelir. Metinler (kelime siralari) hali hazirda sayi dizilerine cevrilmistir ki bu sayilar sozlukteki belirli kelimeleri temsil ederler.

Asagidaki kod IMDB veri setini bilgisayariniza indirir (ya da daha once indirilmis ise onbellekteki kopyayi kullanir):

In [0]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Bagimsiz degisken `num_words=10000` en cok gorulen ilk 10,000 kelimeyi egitim setinde tutar. Nadir gorulen kelimeler ise veri boyutunu kontrol altinda tutmak icin veri setinden atilir.

## Veriyi inceleyelim

Simdi biraz zaman ayirip verinin yapisini anlayalim. Veri seti onceden islenmistir: her ornek film yorumlarindaki kelimeleri temsil eden bir sayi dizisidir. Her etiket degeri 0 ya da 1 olan bir sayidir. 0 olumsuz yorum anlamina gelirken 1 olumlu yorumlardir.

In [0]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Yorum metinleri her sayinin sozlukteki belirli kelimeyi temsil ettigi sayi dizilerine cevrilmistir. Ilk yorumun nasil gozuktugune bir bakalim:

In [0]:
print(train_data[0])

Film yorumlari farkli uzunlukta olabilir. Asagidaki kod ilk ve ikinci yorumlardaki kelime sayisini gosterir. Sinir aglarina girislerin ayni uzunlukta olmasi gerekir, bu daha sonra ustunden gececegimiz bir konu.

In [0]:
len(train_data[0]), len(train_data[1])

### Sayilari kelimere geri cevirelim

Sayilarin kelimelere cevrilip metni yeniden olusturmayi bilmek faydali olabilir. Burada yardimci bir fonksiyon ile sayi-kelime eslemelerini bulunduran sozluk nesnesini sorgulamasini yapacagiz:

In [0]:
# Kelimeleri sayilara eslestiren bir sozluk
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

Simdi `decode_review` fonksiyonunu kullanarak ilk yorumun metnini gorebiliriz:

In [0]:
decode_review(train_data[0])

## Veriyi hazirlayalim

Yorumlarin -sayi dizilerinin- sinir agina verilmeden once tensorlara donusturulmesi gerekmektedir. Bu donusum birkac yolla yapilabilir:

* 'One-hot encoding' yonteminde oldugu gibi dizileri kelimelerin olusuna gore 0 ve 1'den olusan yoneylere cevirebiliriz. Ornegin, [3, 5] dizisi sadece 3 ve 5 dizininde 1 olan ve geri kalan elemanlari 0'dan olusan 10,000-boyutlu bir yoneye donusecektir. Sonra, bu yoney agimizin reel sayi yoney verisini idare edebilen ilk katmanini -yogun katmani- olusturur. Fakat, bu yontem 'num_words * num_reviews' boyutunda matris kadar hafizaya ihtiyac duyar.

* Kullanabilecegimiz diger bir yontem ise dizileri doldurarak ayni uzunluga getirmektir. Sonra, `max_length * num_reviews` seklinde tensorlari olusturabiliriz. Olusan bu sekli idare edebilen gomulu katmani agimizin ilk katmani olarak kullanabiliriz.

Bu egitimde ikinci yontemi kullanacagiz.

Film yorumlarinin ayni uzunlukta olmasi gerektiginden [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) fonksiyonunu kullanarak uzunluklari standartlastiracagiz:

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

Simdi orneklerimizin uzunluklarina bakalim:

In [0]:
len(train_data[0]), len(train_data[1])

Ve (simdi doldurulmus) ilk yorumu inceleyelim:

In [0]:
print(train_data[0])

## Modeli olusturalim

Sinir agi katmanlarin ust uste eklenmesi ile olusturulur-bunun icin iki temel mimari karar gerekir:

* Modelimizde kac adet katman kullanacagiz?
* Her katmanda kac adet *gizli unite* bulunacak?

Bu ornekte girdi verisi kelime dizinleri dizisinden olusmaktadir. Etiketlerin 0 ya da 1 tahmin etmesi gerekir. Bu problem icin modelimizi olusturalim:

In [0]:
# Girdi sekli film yorumlarinda kullanilan kelime sayisidir (10,000 kelime)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Katmanlar sira ile birbirine eklenerek siniflandiriciyi olusturur:

1. Ilk katman `gomulu` katmandir. Bu katman sayi-kodlu kelime hazinesini alir ve her kelime-dizini icin gomulu yoneye bakar. Bu yoneyler model treni olarak ogrenilir. Yoneyler cikti dizisine bir boyut eklerler. Elde edilen boyutlar: `(toplu, sira, gomulu)`.
2. Sonra, `GlobalAveragePooling1D` katmani sira boyutunda ortalanan her ornek icin sabit-uzunlukta cikti yoneyleri dondurur. Bu modelin degisik uzunluktaki girdileri en basit sekilde halletmesini saglar.
3. Bu sabit-uzunluktaki cikti yoneyi 16 gizli uniteden olusan tamamen-bagli (`Dense`) katmana gonderilir.
4. Son katman tek cikti dugumlu yogun bagli bir katmandir. `Sigmoid` aktiflestirici fonksiyonunu kullanarak elde edilen bu deger, olasiligi ya da guven seviyesini temsil eden 0 ile 1 arasinda bir reel sayidir.

### Gizli uniteler

Yukaridaki model girdi ve cikti arasinda iki ara ya da "gizli" katman bulundurmaktadir. Cikti miktari (uniteler, dugumler ya da sinirler) katmani temsil eden boyuttur. Yani icsel temsil edilmeyi ogrenirken sinir aginin sahip oldugu ozgurluktur.

Eger bir model daha cok gizli uniteye (yani daha cok boyutlu temsilsel alani) ve/veya katmani varsa, bu sinir agi daha karisik temsillemeler ogrenebilir. Fakat, bu agin daha cok kaynak kullanmasina neden olur ve istenmeyen kaliplarin ogrenilmesine neden olur. Bu kaliplar egitim verisinde iyi sonuclar almamizi saglerken test verisinde ayni performansi gostermezler. Buna, daha sonra detayli olarak gorecegimiz, *asiri uyum* adi verilir.

### Kayip fonksiyonu ve en iyilestirici

Bir modelin egitimi icin kayip fonksiyonuna ve en iyilestiriciye ihtiyaci vardir. Bu ikili siniflandirma problemi oldugu icin ve modelin ciktisi olasilik (sigmoid aktiflestiricili tek-unite katmani) oldugu icin, biz `binary_crossentropy` kayip fonksiyonunu kullanacagiz.

Bu kayip fonksiyonu icin tek secenegimiz degil, ornegin `mean_squared_error` fonksiyonunu da secebilirdik. Fakat, genellikle `binary_crossentropy` olasiliklar soz konusu oldugunda daha iyi sonuclar verir—olasilik dagilimlari arasindaki "uzakliklari" olcer, ya da bizim durumumuzda, tahminlerle gercek degerler arasindaki farki.

Daha sonra, gerileme problemlerini incelerken (bir evin degerini tahmin ederken mesela), bir diger kayip fonksiyonu olan `mean squared error` fonksiyonunu kullanacagiz.

Simdi modelimizi en iyilestirici ve kayip fonksiyonu kullanacak sekilde ayarlayalim:

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Dogrulama setini olusturalim

Modeli egitirken daha once gormedigi veride modelin dogrulugunu olcmek isteriz. Orjinal egitim verisinden 10,000 ornegi ayirarak *dogrulama setini* olusturun (Neden test setini simdi kullanmiyoruz? Amacimiz egitim verisini kullanarak modelimizi olusturmak ve ayarlamak, daha sonrasinda ise test verisini sadece bir kez kullanarak dogrulugunu degerlendirmek).

In [0]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Modeli egitelim

512 orneklik ufak obeklerle modelimizi 40 devirle egitelim. Bu `x_train` ve `y_train` tensorlarindaki butun ornekleri 40 kere yinelemek anlamina gelir. Egitim sirasinda modelin kaybini ve dogrulugunu dogrulama setindeki 10,000 ornekte takip edin:

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## Modeli degerlendirelim

Simdi modelimiz nasil calisiyor bir bakalim. Iki deger dondurulecek. Kayip (hatamizi temsil eden bir sayi, ne kadar dusuk o kadar iyi) ve dogruluk.

In [0]:
results = model.evaluate(test_data, test_labels)

print(results)

Bu basit yontemle yaklasik %87 dogruluk degeri elde ettik. Daha gelismis yontemler kullanarak modelimizin %95 dogruluga ulasmasini saglayabiliriz.

## Zaman icindeki dogruluk ve kayip degisimi grafigini olusturalim

`model.fit()` fonksiyonu icinde egitim sirasinda olanlar sozlugunu barindiran `History` nesnesi dondurur:

In [0]:
history_dict = history.history
history_dict.keys()

Dort giris vardir: egitim ve dogrulama sirasinda takip edilen olcumleri hepsi icin bir tane. Bunlari kullanarak egitim ve dogrulama kayiplarini ve dogruluk degerlerini karsilastirmak icin cizebiliriz:

In [0]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" "mavi nokta" anlamina gelir
plt.plot(epochs, loss, 'bo', label='Training loss')
# b ise "mavi cizgi" yerine kullanilmistir
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # Sekli temizle

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


Bu grafikte, noktalar egitim kaybini ve dogrulugunu temsil ederken, duz cizgiler dogrulama setinin kaybini ve dogrulugunu gosterir.

Her devirde egitim kaybinin nasil *azaldigini* ve egitim dogrulugunun nasil *arttigini* gozlemleyelim. Bu meyil azalma en iyilestirmesi kullanilirken beklenen etkidir. Istenen miktar her yinelemede azami degere indirgenmelidir.

Aynisi dogrulama setinin kaybiu ve dogruluk degeri icin soyleyemeyecegiz. Bu degerler yirmi devir sonrasinda zirveye ulasmis gibi gozukuyor. Bu asiri uyuma bir ornektir: model egitim verisinde hic gormedigi veriye gore daha iyi sonuclar verir. Bu noktadan sonra model fazladan en iyilestirme yapar ve egitim verisine *ozel* fakat test verisine *genellenemeyen* temsillemeleri ogrenir.

Bu gordugumuz ornekte asiri uyumu yaklasik yirmi devir sonrasinda egitimi durdurarak onleyebiliriz. Daha sonra bunu otomatik olarak geri cagirma ile nasil yapacagimizi gorecegiz.